#    Data 620, Project 1
Team 6: Alice Friedman, Jun Pan, Stephen (Scott) Jones, and Jeff Littlejohn
June 19, 2019

## Part 1: Setup

- Install required packages (NetworkX, Matplotlib.pyplot)

## Load Data

- Load data into csv(?) from https://snap.stanford.edu/data/twitter-2010.html

## Data Exploration

Describe data structure and available fields

## Create Graph

To create a social network graph of users with a directed "follow" relationship to official campaign Twitter of Deval Patrick, who ran for office in 2010

- Add "ego" to graph by locating Twitter user Devel Patrick 
- From TwitterID, Deval Patrick has userID @devalpatrick => 61636488
- Add edges based on users who follow @devalpatrick
    - Include as metadata verified status, or other categorical data
- Create list of TwitterIDs for depth=1
- Add depth=2 edges of users who follow users identified in list above

*Does this make sense? Should we add edges in both directions?



## Plot Initial Graph Structure

## Subset Graph

Use core or island methods described in text book to identify communities

## Plot Subsetted Graph

## Analyze Graph

- Graph Diameter
- Avegage centrality measures (betweenness, degree, pagerank)
    - Plot centrality measures?

Then, calculate each of these measures by verified status --> create dataframe

## Conclusions, Discussion

## References

Data source: [Stanford University Twitter Follower Network, 2010](https://snap.stanford.edu/data/twitter-2010.html)